In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('Asteroid_Modified.csv',low_memory=False)

# Test-Train Splitting

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
input_features = df.columns.values.tolist()
input_features.remove('diameter')

In [5]:
X = df[[i for i in input_features]]

In [6]:
y = df[['diameter']]

In [7]:
X_train, X_rem, y_train, y_rem = train_test_split(X, y , random_state=42, train_size=0.7, shuffle=True)

In [8]:
X_valid, X_test, y_valid, y_test = train_test_split(X, y , random_state=42, train_size=0.3, shuffle=True)

In [9]:
X_train.shape

(96345, 17)

# Prediction Metrics

In [10]:
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, median_absolute_error, explained_variance_score, r2_score

In [11]:
def prediction_metrics(prediction):
    print("Mean Absolute Error :     ",mean_absolute_error(y_test, prediction))
    print("Mean Squared Error :      ",mean_squared_error(y_test, prediction))
    print("Median Absolute Error :   ",median_absolute_error(y_test, prediction))
    print("Explained Variance Score :",explained_variance_score(y_test, prediction))
    print("r2-Score :                ",r2_score(y_test, prediction))

In [12]:
def cv_display_scores(scores):
    print("Scores: ", scores)
    print("Mean:              ", scores.mean())
    print("Standard deviation:", scores.std())

# Neural Network

In [13]:
import tensorflow as tf
from tensorflow import keras

2022-10-28 16:59:32.428815: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## DNN 1

In [14]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=X_train.shape[1:]):
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
        options = {}
    model.add(keras.layers.Dense(1, activation="linear", **options))
    lr_adp = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate,decay_steps=100,decay_rate=0.9)
    optimizer = keras.optimizers.SGD(lr_adp,momentum=0.9)
    model.compile(loss="mean_absolute_error", optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

In [15]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/var/folders/hh/zq1_k2t14hd7ykvvgsbb6f600000gn/T/ipykernel_22583/1709004121.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


In [16]:
keras_reg.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=6)])

2022-10-28 16:59:45.833138: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
3011/3011 [==============================] - 9s 3ms/step - loss: 1274479.8750 - mean_absolute_error: 1274479.8750 - val_loss: 2.5538 - val_mean_absolute_error: 2.5538
Epoch 2/100
3011/3011 [==============================] - 9s 3ms/step - loss: 2.5495 - mean_absolute_error: 2.5495 - val_loss: 2.5419 - val_mean_absolute_error: 2.5419
Epoch 3/100
3011/3011 [==============================] - 7s 2ms/step - loss: 2.5413 - mean_absolute_error: 2.5413 - val_loss: 2.5410 - val_mean_absolute_error: 2.5410
Epoch 4/100
3011/3011 [==============================] - 7s 2ms/step - loss: 2.5407 - mean_absolute_error: 2.5407 - val_loss: 2.5410 - val_mean_absolute_error: 2.5410
Epoch 5/100
3011/3011 [==============================] - 7s 2ms/step - loss: 2.5407 - mean_absolute_error: 2.5407 - val_loss: 2.5410 - val_mean_absolute_error: 2.5410
Epoch 6/100
3011/3011 [==============================] - 6s 2ms/step - loss: 2.5407 - mean_absolute_error: 2.5407 - val_loss: 2.5410 - val_mean_absolute_

In [17]:
y_pred = keras_reg.predict(X_test)

3011/3011 [==============================] - 4s 1ms/step


In [18]:
y_pred

array([3.9322813, 3.9322813, 3.9322813, ..., 3.9322813, 3.9322813,
       3.9322813], dtype=float32)

In [19]:
prediction_metrics(y_pred)

Mean Absolute Error :      2.5385208360572893
Mean Squared Error :       64.39782464999193
Median Absolute Error :    1.3692812557220457
Explained Variance Score : 0.33302221172337076
r2-Score :                 0.3200793609938918


## DNN 2

In [20]:
def build_model2(n_hidden=1, n_neurons=30, learning_rate=0.05, input_shape=X_train.shape[1:]):
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
        options = {}
    model.add(keras.layers.Dense(1, activation="linear", **options))
    lr_adp = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate,decay_steps=100,decay_rate=0.9)
    optimizer = keras.optimizers.SGD(lr_adp,momentum=0.9)
    model.compile(loss="mean_absolute_error", optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

In [21]:
keras_reg2 = keras.wrappers.scikit_learn.KerasRegressor(build_model2)

/var/folders/hh/zq1_k2t14hd7ykvvgsbb6f600000gn/T/ipykernel_22583/1311852810.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg2 = keras.wrappers.scikit_learn.KerasRegressor(build_model2)


In [22]:
keras_reg2.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=6)])

Epoch 1/100
3011/3011 [==============================] - 7s 2ms/step - loss: 1471209728.0000 - mean_absolute_error: 1471209728.0000 - val_loss: 2.7644 - val_mean_absolute_error: 2.7644
Epoch 2/100
3011/3011 [==============================] - 6s 2ms/step - loss: 2.7590 - mean_absolute_error: 2.7590 - val_loss: 2.7644 - val_mean_absolute_error: 2.7644
Epoch 3/100
3011/3011 [==============================] - 6s 2ms/step - loss: 2.7590 - mean_absolute_error: 2.7590 - val_loss: 2.7644 - val_mean_absolute_error: 2.7644
Epoch 4/100
3011/3011 [==============================] - 6s 2ms/step - loss: 2.7590 - mean_absolute_error: 2.7590 - val_loss: 2.7644 - val_mean_absolute_error: 2.7644
Epoch 5/100
3011/3011 [==============================] - 6s 2ms/step - loss: 2.7590 - mean_absolute_error: 2.7590 - val_loss: 2.7644 - val_mean_absolute_error: 2.7644
Epoch 6/100
3011/3011 [==============================] - 6s 2ms/step - loss: 2.7590 - mean_absolute_error: 2.7590 - val_loss: 2.7644 - val_mean_abs

In [23]:
y_pred2 = keras_reg2.predict(X_test)

3011/3011 [==============================] - 4s 1ms/step


In [24]:
y_pred2

array([3.955882, 3.955882, 3.955882, ..., 3.955882, 3.955882, 3.955882],
      dtype=float32)

In [25]:
prediction_metrics(y_pred2)

Mean Absolute Error :      2.7597784843957167
Mean Squared Error :       97.03110376805024
Median Absolute Error :    1.3771179275512697
Explained Variance Score : 0.0
r2-Score :                 -0.024467059811607905


## DNN 3

In [26]:
def build_model3(n_hidden=1, n_neurons=25, learning_rate=1e-3, input_shape=X_train.shape[1:]):
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
        options = {}
    model.add(keras.layers.Dense(1, activation="linear", **options))
    lr_adp = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate,decay_steps=100,decay_rate=0.99)
    optimizer = keras.optimizers.SGD(lr_adp,momentum=0.9999)
    model.compile(loss="mean_absolute_error", optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

In [27]:
keras_reg3 = keras.wrappers.scikit_learn.KerasRegressor(build_model3)

/var/folders/hh/zq1_k2t14hd7ykvvgsbb6f600000gn/T/ipykernel_22583/3164829974.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg3 = keras.wrappers.scikit_learn.KerasRegressor(build_model3)


In [28]:
keras_reg3.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=6)])

Epoch 1/100
3011/3011 [==============================] - 7s 2ms/step - loss: 573010.0000 - mean_absolute_error: 573010.0000 - val_loss: 3.5316 - val_mean_absolute_error: 3.5316
Epoch 2/100
3011/3011 [==============================] - 6s 2ms/step - loss: 3.5724 - mean_absolute_error: 3.5724 - val_loss: 3.6157 - val_mean_absolute_error: 3.6157
Epoch 3/100
3011/3011 [==============================] - 7s 2ms/step - loss: 3.4677 - mean_absolute_error: 3.4677 - val_loss: 3.2657 - val_mean_absolute_error: 3.2657
Epoch 4/100
3011/3011 [==============================] - 7s 2ms/step - loss: 3.3462 - mean_absolute_error: 3.3462 - val_loss: 3.1476 - val_mean_absolute_error: 3.1476
Epoch 5/100
3011/3011 [==============================] - 6s 2ms/step - loss: 3.3330 - mean_absolute_error: 3.3330 - val_loss: 3.3201 - val_mean_absolute_error: 3.3201
Epoch 6/100
3011/3011 [==============================] - 6s 2ms/step - loss: 3.2284 - mean_absolute_error: 3.2284 - val_loss: 3.4146 - val_mean_absolute_er

In [29]:
y_pred3 = keras_reg3.predict(X_test)

3011/3011 [==============================] - 4s 1ms/step


In [30]:
y_pred3

array([4.500564, 4.500564, 4.500564, ..., 4.500564, 4.500564, 4.500564],
      dtype=float32)

In [31]:
prediction_metrics(y_pred3)

Mean Absolute Error :      2.8136640159658874
Mean Squared Error :       95.66945433705428
Median Absolute Error :    1.5865640983581542
Explained Variance Score : 0.0
r2-Score :                 -0.010090587372407311


## DNN 4

In [32]:
def build_model4(n_hidden=1, n_neurons=50, learning_rate=0.01, input_shape=X_train.shape[1:]):
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
        options = {}
    model.add(keras.layers.Dense(1, activation="linear", **options))
    lr_adp = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate,decay_steps=100,decay_rate=0.9)
    optimizer = keras.optimizers.SGD(lr_adp,momentum=0.999)
    model.compile(loss="mean_absolute_error", optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

In [33]:
keras_reg4 = keras.wrappers.scikit_learn.KerasRegressor(build_model4)

/var/folders/hh/zq1_k2t14hd7ykvvgsbb6f600000gn/T/ipykernel_22583/3055908056.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg4 = keras.wrappers.scikit_learn.KerasRegressor(build_model4)


In [34]:
keras_reg4.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=6)])

Epoch 1/100
3011/3011 [==============================] - 7s 2ms/step - loss: 48590248.0000 - mean_absolute_error: 48590248.0000 - val_loss: 3.5975 - val_mean_absolute_error: 3.5975
Epoch 2/100
3011/3011 [==============================] - 8s 3ms/step - loss: 3.1440 - mean_absolute_error: 3.1440 - val_loss: 2.7704 - val_mean_absolute_error: 2.7704
Epoch 3/100
3011/3011 [==============================] - 7s 2ms/step - loss: 2.8584 - mean_absolute_error: 2.8584 - val_loss: 2.7666 - val_mean_absolute_error: 2.7666
Epoch 4/100
3011/3011 [==============================] - 7s 2ms/step - loss: 2.7751 - mean_absolute_error: 2.7751 - val_loss: 2.7892 - val_mean_absolute_error: 2.7892
Epoch 5/100
3011/3011 [==============================] - 9s 3ms/step - loss: 2.7816 - mean_absolute_error: 2.7816 - val_loss: 2.7894 - val_mean_absolute_error: 2.7894
Epoch 6/100
3011/3011 [==============================] - 7s 2ms/step - loss: 2.7815 - mean_absolute_error: 2.7815 - val_loss: 2.7893 - val_mean_absolut

In [35]:
y_pred4 = keras_reg4.predict(X_test)

3011/3011 [==============================] - 4s 1ms/step


In [36]:
y_pred4

array([3.6142397, 3.6142397, 3.6142397, ..., 3.6142397, 3.6142397,
       3.6142397], dtype=float32)

In [37]:
prediction_metrics(y_pred4)

Mean Absolute Error :      2.780967273249466
Mean Squared Error :       98.18798080132058
Median Absolute Error :    1.2982396926879884
Explained Variance Score : 0.0
r2-Score :                 -0.03668151854507906


## DNN 5

In [38]:
def build_model5(n_hidden=1, n_neurons=50, learning_rate=0.1, input_shape=X_train.shape[1:]):
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
        options = {}
    model.add(keras.layers.Dense(1, activation="linear", **options))
    lr_adp = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate,decay_steps=100,decay_rate=0.99)
    optimizer = keras.optimizers.SGD(lr_adp,momentum=0.999)
    model.compile(loss="mean_absolute_error", optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

In [39]:
keras_reg5 = keras.wrappers.scikit_learn.KerasRegressor(build_model5)

/var/folders/hh/zq1_k2t14hd7ykvvgsbb6f600000gn/T/ipykernel_22583/716683303.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg5 = keras.wrappers.scikit_learn.KerasRegressor(build_model5)


In [40]:
keras_reg5.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=6)])

Epoch 1/100
3011/3011 [==============================] - 7s 2ms/step - loss: 7929648128.0000 - mean_absolute_error: 7929648128.0000 - val_loss: 2.7714 - val_mean_absolute_error: 2.7714
Epoch 2/100
3011/3011 [==============================] - 6s 2ms/step - loss: 3.2095 - mean_absolute_error: 3.2095 - val_loss: 2.7760 - val_mean_absolute_error: 2.7760
Epoch 3/100
3011/3011 [==============================] - 7s 2ms/step - loss: 2.9684 - mean_absolute_error: 2.9684 - val_loss: 4.2349 - val_mean_absolute_error: 4.2349
Epoch 4/100
3011/3011 [==============================] - 6s 2ms/step - loss: 3.1799 - mean_absolute_error: 3.1799 - val_loss: 2.7994 - val_mean_absolute_error: 2.7994
Epoch 5/100
3011/3011 [==============================] - 6s 2ms/step - loss: 2.8494 - mean_absolute_error: 2.8494 - val_loss: 2.7644 - val_mean_absolute_error: 2.7644
Epoch 6/100
3011/3011 [==============================] - 6s 2ms/step - loss: 2.8304 - mean_absolute_error: 2.8304 - val_loss: 2.7822 - val_mean_abs

In [41]:
y_pred5 = keras_reg5.predict(X_test)

3011/3011 [==============================] - 4s 1ms/step


In [42]:
y_pred5

array([3.69353, 3.69353, 3.69353, ..., 3.69353, 3.69353, 3.69353],
      dtype=float32)

In [43]:
prediction_metrics(y_pred5)

Mean Absolute Error :      2.77198418081098
Mean Squared Error :       97.89868392560474
Median Absolute Error :    1.3115300827026366
Explained Variance Score : 0.0
r2-Score :                 -0.03362708436708872


# Randomized Search CV

In [44]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [45]:
param_distr = { "n_hidden": [0, 1, 2, 3], "n_neurons": np.arange(1, 100), "learning_rate": reciprocal(3e-4, 3e-2)}

In [46]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distr, n_iter=10, cv=3)

In [47]:
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=4)])

Epoch 1/100
2008/2008 [==============================] - 5s 2ms/step - loss: 599927616.0000 - mean_absolute_error: 599927616.0000 - val_loss: 242112464.0000 - val_mean_absolute_error: 242112464.0000
Epoch 2/100
2008/2008 [==============================] - 5s 2ms/step - loss: 67895304.0000 - mean_absolute_error: 67895304.0000 - val_loss: 8420909.0000 - val_mean_absolute_error: 8420909.0000
Epoch 3/100
2008/2008 [==============================] - 5s 2ms/step - loss: 8179938.5000 - mean_absolute_error: 8179938.5000 - val_loss: 725074.0625 - val_mean_absolute_error: 725074.0625
Epoch 4/100
2008/2008 [==============================] - 5s 2ms/step - loss: 962221.7500 - mean_absolute_error: 962221.7500 - val_loss: 608339.3750 - val_mean_absolute_error: 608339.3750
Epoch 5/100
2008/2008 [==============================] - 5s 2ms/step - loss: 100159.2422 - mean_absolute_error: 100159.2422 - val_loss: 21538.0430 - val_mean_absolute_error: 21538.0430
Epoch 6/100
2008/2008 [========================

2008/2008 [==============================] - 5s 3ms/step - loss: 331186.4375 - mean_absolute_error: 331186.4375 - val_loss: 2.5972 - val_mean_absolute_error: 2.5972
Epoch 2/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.5767 - mean_absolute_error: 2.5767 - val_loss: 2.5710 - val_mean_absolute_error: 2.5710
Epoch 3/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.5627 - mean_absolute_error: 2.5627 - val_loss: 2.5683 - val_mean_absolute_error: 2.5683
Epoch 4/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.5618 - mean_absolute_error: 2.5618 - val_loss: 2.5680 - val_mean_absolute_error: 2.5680
Epoch 5/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.5615 - mean_absolute_error: 2.5615 - val_loss: 2.5680 - val_mean_absolute_error: 2.5680
Epoch 6/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.5615 - mean_absolute_error: 2.5615 - val_loss: 2.5680 - val_mean_absolute_error: 2.5680


2008/2008 [==============================] - 5s 2ms/step - loss: 2.8006 - mean_absolute_error: 2.8006 - val_loss: 2.8032 - val_mean_absolute_error: 2.8032
Epoch 6/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.8006 - mean_absolute_error: 2.8006 - val_loss: 2.8032 - val_mean_absolute_error: 2.8032
Epoch 7/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.8006 - mean_absolute_error: 2.8006 - val_loss: 2.8032 - val_mean_absolute_error: 2.8032
Epoch 8/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.8006 - mean_absolute_error: 2.8006 - val_loss: 2.8032 - val_mean_absolute_error: 2.8032
Epoch 9/100
1004/1004 [==============================] - 2s 2ms/step - loss: 2.7834 - mean_absolute_error: 2.7834
Epoch 1/100
2008/2008 [==============================] - 6s 3ms/step - loss: 13.1436 - mean_absolute_error: 13.1436 - val_loss: 2.8296 - val_mean_absolute_error: 2.8296
Epoch 2/100
2008/2008 [==============================] - 5s 2

Epoch 6/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.5321 - mean_absolute_error: 2.5321 - val_loss: 2.5409 - val_mean_absolute_error: 2.5409
Epoch 7/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.5320 - mean_absolute_error: 2.5320 - val_loss: 2.5409 - val_mean_absolute_error: 2.5409
Epoch 8/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.5320 - mean_absolute_error: 2.5320 - val_loss: 2.5409 - val_mean_absolute_error: 2.5409
Epoch 9/100
2008/2008 [==============================] - 5s 2ms/step - loss: 2.5320 - mean_absolute_error: 2.5320 - val_loss: 2.5409 - val_mean_absolute_error: 2.5409
Epoch 10/100
1004/1004 [==============================] - 2s 2ms/step - loss: 2.5504 - mean_absolute_error: 2.5504
Epoch 1/100
2008/2008 [==============================] - 6s 3ms/step - loss: 15447720.0000 - mean_absolute_error: 15447720.0000 - val_loss: 2.6045 - val_mean_absolute_error: 2.6045
Epoch 2/100
2008/2008 [=============

2008/2008 [==============================] - 6s 3ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/100
2008/2008 [==============================] - 5s 3ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/100
2008/2008 [==============================] - 5s 3ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/100
1004/1004 [==============================] - 2s 2ms/step - loss: nan - mean_absolute_error: nan
Epoch 1/100
2008/2008 [==============================] - 6s 3ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/100
2008/2008 [==============================] - 5s 3ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/100
2008/2008 [==============================] - 5s 3ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val

2008/2008 [==============================] - 5s 2ms/step - loss: 2.5621 - mean_absolute_error: 2.5621 - val_loss: 2.5725 - val_mean_absolute_error: 2.5725
Epoch 8/100
2008/2008 [==============================] - 6s 3ms/step - loss: 2.5621 - mean_absolute_error: 2.5621 - val_loss: 2.5725 - val_mean_absolute_error: 2.5725
Epoch 9/100
2008/2008 [==============================] - 8s 4ms/step - loss: 2.5621 - mean_absolute_error: 2.5621 - val_loss: 2.5725 - val_mean_absolute_error: 2.5725
Epoch 10/100
2008/2008 [==============================] - 6s 3ms/step - loss: 2.5621 - mean_absolute_error: 2.5621 - val_loss: 2.5725 - val_mean_absolute_error: 2.5725
Epoch 11/100
1004/1004 [==============================] - 2s 2ms/step - loss: 2.5729 - mean_absolute_error: 2.5729
Epoch 1/100
2008/2008 [==============================] - 7s 3ms/step - loss: 380633.4375 - mean_absolute_error: 380633.4375 - val_loss: 2.6367 - val_mean_absolute_error: 2.6367
Epoch 2/100
2008/2008 [============================

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_search.py:953: UserWarning: One or more of the test scores are non-finite: [-1.91272474e+04 -2.56479327e+00 -2.79366128e+00 -2.60708920e+00
 -2.62507176e+00 -2.86290606e+00             nan             nan
             nan -2.57551678e+00]
  warnings.warn(


3011/3011 [==============================] - 8s 3ms/step - loss: 196970.6719 - mean_absolute_error: 196970.6719 - val_loss: 2.5645 - val_mean_absolute_error: 2.5645
Epoch 2/100
3011/3011 [==============================] - 9s 3ms/step - loss: 2.5550 - mean_absolute_error: 2.5550 - val_loss: 2.5559 - val_mean_absolute_error: 2.5559
Epoch 3/100
3011/3011 [==============================] - 8s 3ms/step - loss: 2.5511 - mean_absolute_error: 2.5511 - val_loss: 2.5557 - val_mean_absolute_error: 2.5557
Epoch 4/100
3011/3011 [==============================] - 8s 3ms/step - loss: 2.5509 - mean_absolute_error: 2.5509 - val_loss: 2.5557 - val_mean_absolute_error: 2.5557
Epoch 5/100
3011/3011 [==============================] - 8s 3ms/step - loss: 2.5509 - mean_absolute_error: 2.5509 - val_loss: 2.5557 - val_mean_absolute_error: 2.5557
Epoch 6/100
3011/3011 [==============================] - 7s 2ms/step - loss: 2.5509 - mean_absolute_error: 2.5509 - val_loss: 2.5557 - val_mean_absolute_error: 2.5557


RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x1331d2f80>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x133825450>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [48]:
rnd_search_cv.best_params_

{'learning_rate': 0.0006624546927521231, 'n_hidden': 1, 'n_neurons': 36}

In [49]:
rnd_search_cv.best_score_

-2.564793268839518

In [50]:
nn_model = rnd_search_cv.best_estimator_.model

In [51]:
y_pred_best = nn_model.predict(X_test)

3011/3011 [==============================] - 4s 1ms/step


In [52]:
y_pred_best

array([[3.5938022],
       [3.5938022],
       [3.5938022],
       ...,
       [3.5938022],
       [3.5938022],
       [3.5938022]], dtype=float32)

In [53]:
prediction_metrics(y_pred_best)

Mean Absolute Error :      2.5484457546463024
Mean Squared Error :       75.40833696900332
Median Absolute Error :    1.2918022136688232
Explained Variance Score : 0.2249331798100721
r2-Score :                 0.20382893464151997


# Inference

The best model was the Random Forest Regressor with the following statistics:

### Cross Validation:
* Scores:  0.68802357, 0.28007676, 0.06561844, 0.0558667,  0.04707463, 0.03946168, 0.03526181, 0.04808714, 0.06202095, 0.06453636
* Mean:               0.13860280285633117
* Standard deviation: 0.19557082408696733


### Test Data:
* Mean Absolute Error :      0.04189141065507549
* Mean Squared Error :       0.06317341583802455
* Median Absolute Error :    0.02113960507459816
* Explained Variance Score : 0.9432657426983421
* r2-Score :                 0.9432657325718565